In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# NearestNeighbor

Surrogate model based on the N-Dimensional Interpolation library by Stephen Marone.

Here is a simple example where the *NearestNeighbor* surrogate is used to approximate the output of a sinusoidal component.

In [ ]:
import numpy as np

import openmdao.api as om

prob = om.Problem()

sin_mm = om.MetaModelUnStructuredComp()
sin_mm.add_input('x', 2.1)
sin_mm.add_output('f_x', 0., surrogate=om.NearestNeighbor(interpolant_type='linear'))

prob.model.add_subsystem('sin_mm', sin_mm)

prob.setup()

# train the surrogate and check predicted value
sin_mm.options['train_x'] = np.linspace(0,10,20)
sin_mm.options['train_f_x'] = .5*np.sin(sin_mm.options['train_x'])

prob.set_val('sin_mm.x', 2.1)

prob.run_model()

print(prob.get_val('sin_mm.f_x'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob.get_val('sin_mm.f_x'), .5*np.sin(prob.get_val('sin_mm.x')), 2e-3)

## NearestNeighbor Options

All options can be passed in as arguments or set later by accessing the *options* dictionary.

In [ ]:
om.show_options_table("openmdao.surrogate_models.nearest_neighbor.NearestNeighbor")

Additional interpolant-specific options can be passed in as call arguments.

## NearestNeighbor Constructor

The call signature for the *NearestNeighbor* constructor is:
```{eval-rst}
    .. automethod:: openmdao.surrogate_models.nearest_neighbor.NearestNeighbor.__init__
        :noindex:
```

## NearestNeighbor Option Examples

**interpolant_type**

The NearestNeighbor surrogate allows you to choose from three different interpolant types.

| Interpolant | Description |
| --- | --- |
| linear | Interpolates values by forming a hyperplane between the points closest to the prescribed inputs |
| weighted | Computes the weights based on the distance and distance effect. |
| rbf | Compactly Supported Radial Basis Function. (Default) |

**rbf interpolator arguments**

When the "interpolant_type" option is set to "rbf", there are some additional arguments that can be used to control the radial basis function interpolant.

For example, here we use the rbf interpolant for our simple sine model and set the number of neighbors ("num_neighbors") to 3.

In [ ]:
import numpy as np

import openmdao.api as om

prob = om.Problem()

sin_mm = om.MetaModelUnStructuredComp()
sin_mm.add_input('x', 2.1)
sin_mm.add_output('f_x', 0., surrogate=om.NearestNeighbor(interpolant_type='rbf', num_neighbors=3))

prob.model.add_subsystem('sin_mm', sin_mm)

prob.setup()

# train the surrogate and check predicted value
sin_mm.options['train_x'] = np.linspace(0,10,20)
sin_mm.options['train_f_x'] = .5*np.sin(sin_mm.options['train_x'])

prob.set_val('sin_mm.x', 2.1)

prob.run_model()

print(prob.get_val('sin_mm.f_x'))

In [ ]:
assert_near_equal(prob.get_val('sin_mm.f_x'), .5*np.sin(prob.get_val('sin_mm.x')), 5e-3)


The following parameters are available to be adjusted:

**num_neighbors** (int)
    
The number of neighbors to use for interpolation.
    
**rbf_family** (int)
    
Specifies the order of the radial basis function to be used.
  
 - -2 uses an 11th order
 - -1 uses a 9th order
 - any value from 0 to 4 uses an order equal to floor((dimensions-1)/2) + (3\*comp) +1